In [12]:
import os
import csv

def compter_panneaux(images_path, labels_path):
    # Dictionnaire pour compter les panneaux
    panneaux_count = {}

    # Liste des fichiers images
    image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]

    # Parcourir chaque image
    for image_file in image_files:
        # Obtenir le nom de base sans l'extension
        base_name = os.path.splitext(image_file)[0]
        
        # Chemin vers le fichier label correspondant
        label_file = os.path.join(labels_path, base_name + '.csv')
        
        # Vérifier si le fichier label existe et n'est pas vide
        if os.path.exists(label_file) and os.path.getsize(label_file) > 0:
            # Lire le fichier CSV
            with open(label_file, 'r') as csvfile:
                csvreader = csv.reader(csvfile)
                # Parcourir chaque ligne du fichier CSV
                for row in csvreader:
                    if len(row) >= 5:  # Vérifier que la ligne a au moins 5 colonnes
                        # Supposons que la classe du panneau est dans la 5ème colonne (index 4)
                        class_name = row[4]
                        if class_name in panneaux_count:
                            panneaux_count[class_name] += 1
                        else:
                            panneaux_count[class_name] = 1

    # Afficher le résultat
    for panneau, count in panneaux_count.items():
        print(f"Nombre de panneaux '{panneau}' : {count}")

# Chemin vers le dossier des images et des labels
images_path = 'train/images'
labels_path = 'train/labels'

# Appeler la fonction
compter_panneaux(images_path, labels_path)


Nombre de panneaux 'frouge' : 81
Nombre de panneaux 'ceder' : 120
Nombre de panneaux 'interdiction' : 289
Nombre de panneaux 'fvert' : 94
Nombre de panneaux 'stop' : 99
Nombre de panneaux 'danger' : 155
Nombre de panneaux 'obligation' : 117
Nombre de panneaux 'forange' : 54
Nombre de panneaux 'ff' : 10
Nombre de panneaux 'Stop' : 1


In [17]:
import os
import csv

def add_empty_label_to_empty_csv_files(labels_path):
    # Liste des fichiers CSV dans le répertoire des labels
    csv_files = [f for f in os.listdir(labels_path) if f.endswith('.csv')]

    for csv_file in csv_files:
        csv_path = os.path.join(labels_path, csv_file)
        # Vérifie si le fichier est vide ou n'a que des nouvelles lignes
        with open(csv_path, 'r') as file:
            content = file.read().strip()

        if content == "" or content == "empty":
            with open(csv_path, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                # Ajoute l'entrée "empty" dans le fichier CSV vide
                csvwriter.writerow(["X", "X", "X", "X", "empty"])
        else:
            # Vérifie et corrige l'annotation "Stop"
            with open(csv_path, 'r') as csvfile:
                csvreader = csv.reader(csvfile)
                rows = list(csvreader)
                for row in rows:
                    if len(row) > 4 and row[4] == "Stop":
                        row[4] = "stop"

            # Écrit les lignes corrigées dans le fichier CSV
            with open(csv_path, 'w', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                csvwriter.writerows(rows)

# Appeler la fonction pour ajouter l'étiquette "empty"
add_empty_label_to_empty_csv_files(labels_path)

def count_empty_labels(labels_path):
    empty_count = 0

    # Liste des fichiers CSV dans le répertoire des labels
    csv_files = [f for f in os.listdir(labels_path) if f.endswith('.csv')]

    for csv_file in csv_files:
        csv_path = os.path.join(labels_path, csv_file)
        with open(csv_path, 'r') as csvfile:
            csvreader = csv.reader(csvfile)
            rows = list(csvreader)
            # Vérifie si le fichier ne contient que l'étiquette "empty"
            if len(rows) == 1 and rows[0][4] == "empty":
                empty_count += 1

    return empty_count

# Chemin vers le dossier des labels
labels_path = 'train/labels2'

# Compter le nombre de fichiers CSV contenant l'étiquette "empty"
empty_files_count = count_empty_labels(labels_path)
print(f"Nombre de fichiers CSV 'empty' : {empty_files_count}")

images_path2 = 'train/images2'
labels_path2 = 'train/labels2'
compter_panneaux(images_path2, labels_path2)


Nombre de fichiers CSV 'empty' : 54
Nombre de panneaux 'frouge' : 81
Nombre de panneaux 'ceder' : 120
Nombre de panneaux 'interdiction' : 289
Nombre de panneaux 'fvert' : 94
Nombre de panneaux 'stop' : 100
Nombre de panneaux 'danger' : 155
Nombre de panneaux 'obligation' : 117
Nombre de panneaux 'empty' : 54
Nombre de panneaux 'forange' : 54
Nombre de panneaux 'ff' : 10



def clear_empty_label_files(labels_path):
    # Liste des fichiers CSV dans le répertoire des labels
    csv_files = [f for f in os.listdir(labels_path) if f.endswith('.csv')]

    for csv_file in csv_files:
        csv_path = os.path.join(labels_path, csv_file)
        # Vérifie si le fichier est vide ou n'a que des nouvelles lignes
        with open(csv_path, 'r') as file:
            content = file.read().strip()
        if  content == "empty":
            with open(csv_path, 'w', newline='') as csvfile:
                 csvfile.truncate()

clear_empty_label_files(labels_path)

# Chemin vers le dossier des labels
labels_path = 'train/labels'

In [25]:
import os
import csv
import shutil
import random
import numpy as np
from PIL import Image, ImageEnhance, ImageOps

def add_noise(img, noise_level=0.05):
    """Add random noise to an image."""
    np_img = np.array(img)
    noise = np.random.normal(loc=0, scale=noise_level, size=np_img.shape)
    np_img = np.clip(np_img + noise * 255, 0, 255).astype(np.uint8)
    return Image.fromarray(np_img)

def rotate_image_left(img):
    """Rotate the image 90 degrees to the left (counter-clockwise)."""
    return img.transpose(Image.ROTATE_90)

def adjust_and_save_image(image_path, csv_path, new_image_path, new_csv_path, transformation):
    """Create an adjusted image and save it with a new label CSV."""
    img = Image.open(image_path)

    # Apply the specified transformation
    if transformation == "rotate_left":
        img = rotate_image_left(img)
    elif transformation == "flip_and_noise":
        img = rotate_image_left(img)  # Mirroring effect
        img = add_noise(img)

    img.save(new_image_path)

    # Copy the CSV file with the new name
    shutil.copy(csv_path, new_csv_path)

def balance_dataset(images_path, labels_path, new_images_path):
    # Create new images and labels directories if they don't exist
    new_images_dir = os.path.join(new_images_path, 'images')
    new_labels_dir = os.path.join(new_images_path, 'labels')
    os.makedirs(new_images_dir, exist_ok=True)
    os.makedirs(new_labels_dir, exist_ok=True)

    # List of types to augment
    types_to_augment = ['frouge', 'ceder', 'fvert', 'stop', 'forange']
    types_to_flip_and_noise = ['frouge', 'fvert', 'forange']

    # Count the number of each type of panel and keep track of produced images
    panneaux_count = {type_name: 0 for type_name in types_to_augment}
    image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]

    for image_file in image_files:
        base_name = os.path.splitext(image_file)[0]
        label_file = os.path.join(labels_path, base_name + '.csv')
        already_augmented = False

        if os.path.exists(label_file) and os.path.getsize(label_file) > 0:
            with open(label_file, 'r') as csvfile:
                csvreader = csv.reader(csvfile)
                rows = list(csvreader)

                # Check if the image contains a unique panel of a type we want to augment
                if len(rows) == 1 and rows[0][4] in types_to_augment:
                    class_name = rows[0][4]

                    # Perform left rotation augmentation
                    new_base_name = f"{base_name}_aug_left"
                    new_image_path = os.path.join(new_images_dir, new_base_name + '.jpg')
                    new_csv_path = os.path.join(new_labels_dir, new_base_name + '.csv')
                    adjust_and_save_image(os.path.join(images_path, image_file), label_file, new_image_path, new_csv_path, "rotate_left")
                    panneaux_count[class_name] += 1
                    already_augmented = True

                    # Perform flip and noise augmentation if applicable
                    if class_name in types_to_flip_and_noise:
                        new_base_name = f"{base_name}_aug_flip_noise"
                        new_image_path = os.path.join(new_images_dir, new_base_name + '.jpg')
                        new_csv_path = os.path.join(new_labels_dir, new_base_name + '.csv')
                        adjust_and_save_image(os.path.join(images_path, image_file), label_file, new_image_path, new_csv_path, "flip_and_noise")
                        panneaux_count[class_name] += 1

        # Perform final augmentation if not already augmented and does not contain 'interdiction'
        if not already_augmented:
            with open(label_file, 'r') as csvfile:
                csvreader = csv.reader(csvfile)
                rows = list(csvreader)
                contains_interdiction = any(row[4] == 'interdiction' for row in rows)
                if not contains_interdiction:
                    new_base_name = f"{base_name}_aug_final"
                    new_image_path = os.path.join(new_images_dir, new_base_name + '.jpg')
                    new_csv_path = os.path.join(new_labels_dir, new_base_name + '.csv')
                    adjust_and_save_image(os.path.join(images_path, image_file), label_file, new_image_path, new_csv_path, "flip_and_noise")

        # Copy original images and labels to the new directories
        shutil.copy(os.path.join(images_path, image_file), os.path.join(new_images_dir, image_file))
        shutil.copy(label_file, os.path.join(new_labels_dir, base_name + '.csv'))

    # Print the result
    for panneau, count in panneaux_count.items():
        print(f"Nombre d'images produites pour '{panneau}' : {count}")

# Définir les chemins
images_path = 'train/images2'
labels_path = 'train/labels2'
new_images_path = 'Newimages'

# Appeler la fonction pour équilibrer le dataset
balance_dataset(images_path, labels_path, new_images_path)


Nombre d'images produites pour 'frouge' : 24
Nombre d'images produites pour 'ceder' : 48
Nombre d'images produites pour 'fvert' : 44
Nombre d'images produites pour 'stop' : 68
Nombre d'images produites pour 'forange' : 34


In [28]:
images_path3 = 'Newimages/images'
labels_path3 = 'Newimages/labels'
compter_panneaux(images_path, labels_path)
print("")
compter_panneaux(images_path3, labels_path3)

Nombre de panneaux 'frouge' : 81
Nombre de panneaux 'ceder' : 120
Nombre de panneaux 'interdiction' : 289
Nombre de panneaux 'fvert' : 94
Nombre de panneaux 'stop' : 100
Nombre de panneaux 'danger' : 155
Nombre de panneaux 'obligation' : 117
Nombre de panneaux 'empty' : 54
Nombre de panneaux 'forange' : 54
Nombre de panneaux 'ff' : 10

Nombre de panneaux 'frouge' : 161
Nombre de panneaux 'ceder' : 211
Nombre de panneaux 'interdiction' : 289
Nombre de panneaux 'fvert' : 188
Nombre de panneaux 'stop' : 176
Nombre de panneaux 'danger' : 247
Nombre de panneaux 'obligation' : 215
Nombre de panneaux 'empty' : 108
Nombre de panneaux 'forange' : 111
Nombre de panneaux 'ff' : 15


In [33]:
import os
import csv
from PIL import Image
import numpy as np
from skimage.feature import hog
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from joblib import Parallel, delayed

def extract_hog_features_single_image(image):
    """Extract HOG features from a single image."""
    image = image.convert('L')  # Convert to grayscale
    image = image.resize((256, 256))  # Resize to 256x256
    image = np.array(image)
    features, _ = hog(image, orientations=9, pixels_per_cell=(8, 8),
                      cells_per_block=(2, 2), block_norm='L2-Hys',
                      visualize=True, transform_sqrt=True)
    return features

def load_data(images_path, labels_path):
    """Load images and labels from the specified directories."""
    images = []
    labels = []
    image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]
    for image_file in image_files:
        image_path = os.path.join(images_path, image_file)
        label_file = os.path.join(labels_path, os.path.splitext(image_file)[0] + '.csv')

        image = Image.open(image_path)
        
        with open(label_file, 'r') as csvfile:
            csvreader = csv.reader(csvfile)
            panel_labels = set()
            for row in csvreader:
                if row[4] != 'empty':  # Skip images labeled as 'empty'
                    panel_labels.add(row[4])
            if panel_labels:
                images.append(image)
                labels.append(panel_labels)

    return images, labels

def extract_hog_features_parallel(images):
    """Extract HOG features from a list of images in parallel."""
    hog_features = Parallel(n_jobs=-1)(delayed(extract_hog_features_single_image)(image) for image in images)
    return hog_features

images_path = 'Newimages/images'
labels_path = 'Newimages/labels'
images, labels = load_data(images_path, labels_path)

# Convertir les ensembles de labels en listes
labels_list = [list(label_set) for label_set in labels]

# Extraire les descripteurs HOG en parallèle
hog_features = extract_hog_features_parallel(images)

# Convert list of features to a numpy array
hog_features = np.array(hog_features)

# Encoder les étiquettes
le = LabelEncoder()
labels_encoded = [le.fit_transform(label_set) for label_set in labels_list]

# Flatten the labels_encoded for fitting
flattened_labels = [item for sublist in labels_encoded for item in sublist]
flattened_hog_features = [hog_features[i] for i, sublist in enumerate(labels_encoded) for _ in sublist]

# Convert flattened_hog_features to a numpy array
flattened_hog_features = np.array(flattened_hog_features)


In [34]:
# Entraîner un SVM avec One-vs-Rest
svm_clf = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
svm_clf.fit(flattened_hog_features, flattened_labels)

# Évaluer le modèle sur les données d'entraînement
predicted_labels = svm_clf.predict(flattened_hog_features)
accuracy = accuracy_score(flattened_labels, predicted_labels)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 80.96%


In [41]:
def load_data(images_path, labels_path):
    """Load images and labels from the specified directories."""
    images = []
    labels = []
    image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]
    for image_file in image_files:
        image_path = os.path.join(images_path, image_file)
        label_file = os.path.join(labels_path, os.path.splitext(image_file)[0] + '.csv')

        image = Image.open(image_path)
        
        with open(label_file, 'r') as csvfile:
            csvreader = csv.reader(csvfile)
            panel_labels = set()
            for row in csvreader:
                if len(row) > 4 and row[4] != 'empty':  # Skip empty labels and check for valid length
                    panel_labels.add(row[4])
            if panel_labels:  # Only add images that have labels
                images.append(image)
                labels.append(panel_labels)

    return images, labels

def detect_panels_in_image(image, svm_clf, le, window_size=(256, 256), step_size=128):
    detected_panels = set()
    image = image.convert('L')  # Convert to grayscale
    image_np = np.array(image)
    for y in range(0, image_np.shape[0] - window_size[1], step_size):
        for x in range(0, image_np.shape[1] - window_size[0], step_size):
            window = image_np[y:y + window_size[1], x:x + window_size[0]]
            window_image = Image.fromarray(window).resize((256, 256))  # Resize window to 256x256
            window_np = np.array(window_image)
            features, _ = hog(window_np, orientations=9, pixels_per_cell=(8, 8),
                              cells_per_block=(2, 2), block_norm='L2-Hys',
                              visualize=True, transform_sqrt=True)
            features = features.reshape(1, -1)
            predictions = svm_clf.predict(features)
            predicted_label = le.inverse_transform(predictions)
            if predicted_label[0] != 'empty':  # Assume 'empty' means no panel
                detected_panels.add(predicted_label[0])
    return detected_panels
# Charger les données de validation
val_images_path = 'val/images'
val_labels_path = 'val/labels'
val_images, val_labels = load_data(val_images_path, val_labels_path)

# Afficher les prédictions pour les 5 premières images de validation
for i in range(min(5, len(val_images))):
    detected_panels = detect_panels_in_image(val_images[i], svm_clf, le)
    print(f"Image {i+1} - Predicted: {detected_panels}, Actual: {val_labels[i]}")

# Évaluer l'accuracy multi-label sur les données de validation
multi_label_accuracy = evaluate_multi_label_accuracy(val_images, val_labels, svm_clf, le)
print(f"Multi-label Validation Accuracy: {multi_label_accuracy * 100:.2f}%")


Image 1 - Predicted: {'fvert'}, Actual: {'frouge', 'interdiction'}
Image 2 - Predicted: {'fvert'}, Actual: {'ceder', 'obligation'}
Image 3 - Predicted: {'fvert'}, Actual: {'stop'}
Image 4 - Predicted: {'fvert'}, Actual: {'frouge', 'interdiction', 'obligation'}
Image 5 - Predicted: {'fvert'}, Actual: {'interdiction'}


KeyboardInterrupt: 

In [ ]:
import os
import csv
import numpy as np
import joblib
from skimage import io, color
from skimage.feature import hog
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from joblib import Parallel, delayed
from skimage.transform import resize

# Chemins vers les dossiers
new_images_path = 'Newimages'
images_path = os.path.join(new_images_path, 'images')
labels_path = os.path.join(new_images_path, 'labels')

# Paramètres HOG
hog_params = {
    'orientations': 9,
    'pixels_per_cell': (8, 8),
    'cells_per_block': (2, 2),
    'block_norm': 'L2-Hys'
}

def extract_hog_features(image_path):
    # Lire l'image
    image = io.imread(image_path)
    # Convertir l'image en niveaux de gris
    gray_image = color.rgb2gray(image)
    # Redimensionner l'image pour une taille fixe (optionnel)
    resized_image = resize(gray_image, (128, 128))
    # Extraire les caractéristiques HOG
    hog_features = hog(resized_image, **hog_params)
    return hog_features

def load_data_parallel(images_path, labels_path, n_jobs=-1):
    image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]
    
    def process_image(image_file):
        base_name = os.path.splitext(image_file)[0]
        label_file = os.path.join(labels_path, base_name + '.csv')
        image_path = os.path.join(images_path, image_file)
        
        if os.path.exists(label_file):
            with open(label_file, 'r') as csvfile:
                csvreader = csv.reader(csvfile)
                labels = [row for row in csvreader if row]
                if labels:
                    for row in labels:
                        if len(row) >= 5:  # Vérifie que la ligne a au moins 5 éléments
                            class_name = row[4]
                            if class_name[0] != "empty":
                                # Extraire les caractéristiques HOG
                                hog_features = extract_hog_features(image_path)
                                return hog_features, class_name
        return None
    
    results = Parallel(n_jobs=n_jobs)(delayed(process_image)(image_file) for image_file in image_files)
    results = [r for r in results if r is not None]
    
    if not results:
        raise ValueError("Aucune donnée valide n'a été trouvée. Vérifiez les fichiers d'entrée.")

    features, labels = zip(*results)
    return np.array(features), np.array(labels)

# Charger les données en parallèle
features, labels = load_data_parallel(images_path, labels_path)

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Entraîner le classifieur SVM
svm_clf = svm.SVC(kernel='linear', probability=True)
svm_clf.fit(X_train, y_train)

# Prédire sur l'ensemble de test
y_pred = svm_clf.predict(X_test)

# Évaluer les performances du classifieur
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred))

# Sauvegarder le modèle entraîné
joblib.dump(svm_clf, 'svm_hog_classifier.pkl')


In [ ]:
import os
import numpy as np
import joblib
from skimage import io
from skimage.feature import hog
from joblib import Parallel, delayed
from skimage.transform import resize
from sklearn.metrics import accuracy_score, classification_report

# Chemins vers les dossiers
val_images_path = os.path.join('val', 'images')
val_labels_path = os.path.join('val', 'labels')
model_path = 'svm_hog_classifier.pkl'

# Paramètres HOG (doivent être les mêmes que ceux utilisés pendant l'entraînement)
hog_params = {
    'orientations': 9,
    'pixels_per_cell': (8, 8),
    'cells_per_block': (2, 2),
    'block_norm': 'L2-Hys'
}

# Taille fixe des images utilisées pour l'extraction des caractéristiques HOG
image_size = (128, 128)

def extract_hog_features(image_path):
    # Lire l'image
    image = io.imread(image_path)
    # Redimensionner l'image pour une taille fixe (doit être la même que celle utilisée pendant l'entraînement)
    resized_image = resize(image, image_size)
    # Extraire les caractéristiques HOG en spécifiant l'axe des canaux
    hog_features = hog(resized_image, channel_axis=-1, **hog_params)
    return hog_features

def load_val_data(images_path, labels_path, n_jobs=-1):
    image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]
    
    def process_image(image_file):
        base_name = os.path.splitext(image_file)[0]
        label_file = os.path.join(labels_path, base_name + '.csv')
        image_path = os.path.join(images_path, image_file)
        
        if os.path.exists(label_file):
            with open(label_file, 'r') as csvfile:
                csvreader = csv.reader(csvfile)
                labels = [row for row in csvreader if row]
                if labels:
                    for row in labels:
                        if len(row) >= 5:  # Vérifie que la ligne a au moins 5 éléments
                            class_name = row[4]
                            if class_name != "empty":
                                # Extraire les caractéristiques HOG
                                hog_features = extract_hog_features(image_path)
                                return hog_features, class_name, image_file
        return None
    
    results = Parallel(n_jobs=n_jobs)(delayed(process_image)(image_file) for image_file in image_files)
    results = [r for r in results if r is not None]
    
    if not results:
        raise ValueError("Aucune donnée valide n'a été trouvée. Vérifiez les fichiers d'entrée.")

    features, labels, image_files = zip(*results)
    return np.array(features), np.array(labels), image_files

# Charger les données de validation
val_features, val_labels, val_image_files = load_val_data(val_images_path, val_labels_path)

# Charger le modèle SVM entraîné
svm_clf = joblib.load(model_path)

# Prédire sur le dataset de validation
val_predictions = svm_clf.predict(val_features)

# Calculer le taux de bons résultats
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Taux de bon résultat : {accuracy * 100:.2f}%")

# Afficher le rapport de classification
print("Classification Report:")
print(classification_report(val_labels, val_predictions))

# Afficher 5 noms d'images avec la classe prédite
print("5 images avec leur classe prédite :")
for i in range(min(5, len(val_image_files))):
    print(f"Image: {val_image_files[i]}, Classe réelle: {val_labels[i]}, Classe prédite: {val_predictions[i]}")


In [ ]:
import os
import numpy as np
import joblib
from skimage import io
from skimage.feature import hog
from joblib import Parallel, delayed
from skimage.transform import resize
from sklearn.metrics import accuracy_score, classification_report

# Chemins vers les dossiers
val_images_path = os.path.join('val', 'images')
val_labels_path = os.path.join('val', 'labels')
model_path = 'svm_hog_classifier.pkl'

# Paramètres HOG (doivent être les mêmes que ceux utilisés pendant l'entraînement)
hog_params = {
    'orientations': 9,
    'pixels_per_cell': (8, 8),
    'cells_per_block': (2, 2),
    'block_norm': 'L2-Hys'
}

# Taille fixe des images utilisées pour l'extraction des caractéristiques HOG
image_size = (256, 256)

def extract_hog_features(image_path):
    # Lire l'image
    image = io.imread(image_path)
    # Redimensionner l'image pour une taille fixe (doit être la même que celle utilisée pendant l'entraînement)
    resized_image = resize(image, image_size)
    # Extraire les caractéristiques HOG en spécifiant l'axe des canaux
    hog_features = hog(resized_image, channel_axis=-1, **hog_params)
    return hog_features

def load_val_data(images_path, labels_path, n_jobs=-1):
    image_files = [f for f in os.listdir(images_path) if f.endswith('.jpg')]
    
    def process_image(image_file):
        base_name = os.path.splitext(image_file)[0]
        label_file = os.path.join(labels_path, base_name + '.csv')
        image_path = os.path.join(images_path, image_file)
        
        if os.path.exists(label_file):
            with open(label_file, 'r') as csvfile:
                csvreader = csv.reader(csvfile)
                labels = [row for row in csvreader if row]
                if labels:
                    for row in labels:
                        if len(row) >= 5:  # Vérifie que la ligne a au moins 5 éléments
                            class_name = row[4]
                            if class_name != "empty":
                                # Extraire les caractéristiques HOG
                                hog_features = extract_hog_features(image_path)
                                return hog_features, class_name, image_file
        return None
    
    results = Parallel(n_jobs=n_jobs)(delayed(process_image)(image_file) for image_file in image_files)
    results = [r for r in results if r is not None]
    
    if not results:
        raise ValueError("Aucune donnée valide n'a été trouvée. Vérifiez les fichiers d'entrée.")

    features, labels, image_files = zip(*results)
    return np.array(features), np.array(labels), image_files

# Charger les données de validation
val_features, val_labels, val_image_files = load_val_data(val_images_path, val_labels_path)

# Charger le modèle SVM entraîné
svm_clf = joblib.load(model_path)

# Prédire sur le dataset de validation
val_predictions = svm_clf.predict(val_features)

# Calculer le taux de bons résultats
accuracy = accuracy_score(val_labels, val_predictions)
print(f"Taux de bon résultat : {accuracy * 100:.2f}%")

# Afficher le rapport de classification
print("Classification Report:")
print(classification_report(val_labels, val_predictions))

# Afficher 5 noms d'images avec la classe prédite
print("5 images avec leur classe prédite :")
for i in range(min(5, len(val_image_files))):
    print(f"Image: {val_image_files[i]}, Classe réelle: {val_labels[i]}, Classe prédite: {val_predictions[i]}")


Utilisateur de OneVsRestClassifier